In [ ]:
# default_exp losses.numpy

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# EVALUATION METRICS

> API details.

In [ ]:
#export
from math import sqrt

import numpy as np

# TODO: Create new file with protections shared by pytorch losses and numpy losses (divide_no_nan)
# TODO: Think more efficient way of masking y_mask availability, without indexing (maybe 0s)
# TODO: Think how to deal with general scaled metrics, like MASE and RMAE

In [ ]:
#export
def divide_no_nan(a, b):
    """
    Auxiliary funtion to handle divide by 0
    """
    div = a / b
    div[div != div] = 0.0
    div[div == float('inf')] = 0.0
    return div

In [ ]:
# export
def metric_protections(y: np.ndarray, y_hat: np.ndarray, weights):
    assert (weights is None) or (np.sum(weights)>0), 'Sum of weights cannot be 0'
    assert (weights is None) or (weights.shape == y.shape), 'Wrong weight dimension'

In [ ]:
#export
def mape(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    #TODO: weights no hace nada
    """Calculates Mean Absolute Percentage Error.
    MAPE measures the relative prediction accuracy of a
    forecasting method by calculating the percentual deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values
    weights: numpy array
      weights for weigted average
 
    Returns
    -------
    scalar: MAPE
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y)
    mape = divide_no_nan(delta_y, scale)
    mape = np.average(mape, weights=weights)
    mape = 100 * mape
    return mape

In [ ]:
#export
def mse(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    """Calculates Mean Squared Error.
    MSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    
    Parameters
    ----------
    y: numpy array
        actual test values
    y_hat: numpy array
        predicted values 
    weights: numpy array
      weights for weigted average
        
    Returns
    -------
    scalar: MSE
    """
    metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
        mse = np.average(delta_y[~np.isnan(delta_y)], weights=weights[~np.isnan(delta_y)])
    else:
        mse = np.nanmean(delta_y)
    return mse

In [ ]:
#export
def rmse(y: np.ndarray, y_hat: np.ndarray, weights=None) -> float:
    """Calculates Root Mean Squared Error.
    RMSE measures the prediction accuracy of a
    forecasting method by calculating the squared deviation
    of the prediction and the true value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values    
    weights: numpy array
      weights for weigted average      
      
    Returns
    -------
    scalar: RMSE
    """

    return np.sqrt(mse(y, y_hat, weights))

In [ ]:
#export
def smape(y: np.ndarray, y_hat: np.ndarray,
          y_mask=None, weights=None) -> float:
    """Calculates Symmetric Mean Absolute Percentage Error.
    
    SMAPE measures the relative prediction accuracy of a
    forecasting method by calculating the relative deviation
    of the prediction and the true value scaled by the sum of the
    absolute values for the prediction and true value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desireble compared to normal MAPE that
    may be undetermined.
    
    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array
      predicted values
    y_mask: numpy array
      optional mask, 1 keep 0 omit      
    weights: numpy array
      weights for weigted average
    
    Returns
    -------
    scalar: SMAPE
    """
    metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = divide_no_nan(delta_y, scale)
    smape = 200 * np.average(smape, weights=weights)

    assert smape <= 200, 'SMAPE should be lower than 200'
    return smape

In [ ]:
#export
def mase(y: np.ndarray, y_hat: np.ndarray, weights=None):
    """ Calculates the M4 Mean Absolute Scaled Error.

    MASE measures the relative prediction accuracy of a
    forecasting method by comparinng the mean absolute errors
    of the prediction and the true value against the mean
    absolute errors of the seasonal naive model.

    Parameters
    ----------
    seasonality: int
        main frequency of the time series
        Hourly 24,  Daily 7, Weekly 52,
        Monthly 12, Quarterly 4, Yearly 1
    y: tensor (batch_size, output_size)
        actual test values
    y_hat: tensor (batch_size, output_size)
        predicted values
    y_train: tensor (batch_size, input_size)
        actual insample values for Seasonal Naive predictions

    Returns
    -------
    mase:
        mean absolute scaled error

    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf
    """    
    print("Not implemented yet")

In [ ]:
#export
def mae(y: np.ndarray, y_hat: np.ndarray, weights=None):
    """Calculates Mean Absolute Error.

    The mean absolute error 

    Parameters
    ----------
    y: numpy array
        actual test values
    y_hat: numpy array
        predicted values
    weights: numpy array
      weights for weigted average

    Return
    ------
    scalar: MAE
    """
    metric_protections(y, y_hat, weights)
    
    delta_y = np.abs(y - y_hat)
    if weights is not None:
        mae = np.average(delta_y[~np.isnan(delta_y)], weights=weights[~np.isnan(delta_y)])
    else:
        mae = np.nanmean(delta_y)
    return mae

In [ ]:
#export
def pinball_loss(y: np.ndarray, y_hat: np.ndarray, tau: float=0.5, weights=None) -> np.ndarray:
    """Calculates the Pinball Loss.

    The Pinball loss measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.
    A common value for tau is 0.5 for the deviation from the median.

    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat: numpy array of len h (forecasting horizon)
      predicted values   
    weights: numpy array
      weights for weigted average      
    tau: float
      Fixes the quantile against which the predictions are compared.
    Return
    ------
    return: pinball_loss
    """
    metric_protections(y, y_hat, weights)

    delta_y = y - y_hat
    pinball = np.maximum(tau * delta_y, (tau - 1) * delta_y)

    if weights is not None:
        pinball = np.average(pinball[~np.isnan(pinball)], weights=weights[~np.isnan(pinball)])
    else:
        pinball = np.nanmean(pinball)
    return pinball

In [ ]:
# export
def rmae(y: np.ndarray, 
         y_hat1: np.ndarray, y_hat2: np.ndarray, 
         weights=None):
    """Calculates Relative Mean Absolute Error.

    The relative mean absolute error of two forecasts.
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator.

    Parameters
    ----------
    y: numpy array
      actual test values
    y_hat1: numpy array
      predicted values of first model
    y_hat2: numpy array
      predicted values of second model
    weights: numpy array
      weights for weigted average
    freq: int
      frequency of the y series, it will determine the
      seasonal naive benchmark

    Return
    ------
    scalar: rMAE
    """
    numerator = mae(y=y, y_hat=y_hat1, weights=weights)
    denominator = mae(y=y, y_hat=y_hat2, weights=weights)
    rmae = numerator/denominator
    return rmae

# MULTI-QUANTILE LOSS

MQLoss definition and testing

In [ ]:
#export
def mqloss(y, y_hat, quantiles, weights=None): 
    """mqloss

    Calculates Average Multi-quantile Loss function, for
    a given set of quantiles, based on the absolute 
    difference between predicted and true values.

    Parameters
    ----------
    y: np.array (batch_size, output_size) actual values in torch tensor.
    y_hat: np.array (batch_size, output_size, n_quantiles) predicted values in torch tensor.
    mask: np.array (batch_size, output_size, n_quantiles) specifies date stamps per serie
          to consider in loss
    quantiles: np.array(n_quantiles) quantiles to estimate from the distribution of y.

    Returns
    -------
    lq: np.array(n_quantiles) average multi-quantile loss.
    """    
    
    if weights is None: weights = np.ones_like(y)

    n_q = len(quantiles)
    
    y_rep = np.expand_dims(y, axis=-1)
    error = y_hat - y_rep
    sq = np.maximum(-error, np.zeros_like(error))
    s1_q = np.maximum(error, np.zeros_like(error)) 
    loss = (quantiles * sq + (1 - quantiles) * s1_q) * np.expand_dims(weights, axis=-1)
    return np.mean(np.mean(loss, axis=1))

In [ ]:
y = np.random.random(size=(100, 7))
y_q = np.random.random(size=(100, 7, 4))
weights = np.ones_like(y)
quantiles = np.array([0.1, 0.2, 0.3, 0.4])

mqloss(y, y_q, quantiles, weights)

0.17958933926299467

In [ ]:
mqloss(y, y_q, quantiles)

0.17958933926299467

In [ ]:
#export
def wmqloss(y, y_hat, quantiles, weights=None): 
    """wmqloss

    Calculates Average Multi-quantile Loss function, for
    a given set of quantiles, based on the absolute 
    difference between predicted and true values.

    Parameters
    ----------
    y: np.array (batch_size, output_size) actual values in torch tensor.
    y_hat: np.array (batch_size, output_size, n_quantiles) predicted values in torch tensor.
    mask: np.array (batch_size, output_size, n_quantiles) specifies date stamps per serie
          to consider in loss
    quantiles: np.array(n_quantiles) quantiles to estimate from the distribution of y.

    Returns
    -------
    lq: np.array(n_quantiles) average multi-quantile loss.
    """    
    
    if weights is None: weights = np.ones_like(y)

        
    n_q = len(quantiles)
    
    y_rep = np.expand_dims(y, axis=-1)
    error = y_hat - y_rep
    sq = np.maximum(-error, np.zeros_like(error))
    s1_q = np.maximum(error, np.zeros_like(error)) 
    loss = (quantiles * sq + (1 - quantiles) * s1_q)
    
    w = np.expand_dims(weights, axis=-1)
    loss = divide_no_nan(np.sum(loss * w, axis=-2), 
                         np.sum(np.abs(y_rep) * w, axis=-2))
    
    return np.mean(loss)

In [ ]:
wmqloss(y, y_q, quantiles, weights)

0.4205550107407827

In [ ]:
wmqloss(y, y_q, quantiles)

0.4205550107407827

# TEST/DEBUG NUMPY EVALUATION METRICS

In [ ]:
y = np.array([1,1,1,0,0,0,0,0,1, np.nan])
y_mask = np.array([1,1,1,1,1,1,1,1,2,0])
y_hat = np.array([1,2,3,-4,-5,-6,-7,-8,-9,-10])

print(mae(y=y, y_hat=y_hat, weights=y_mask))
print(mae(y=y, y_hat=y_hat))

5.3
4.777777777777778


In [ ]:
print(mae(y=y, y_hat=y_hat, weights=y_mask))
print(mae(y=y, y_hat=y_hat))

5.3
4.777777777777778


In [ ]:
len(y)

10